In [1]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import wavy
from wavy.models import compute_default_scores
from wavy.utils import reverse_pct_change

/Users/rodrigonader/anaconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [4]:
msft = yf.Ticker("MSFT")

In [5]:
# get historical market data from 2005 on
hist = msft.history(period="max", start="2005-01-01")

In [6]:
hist[['Open', 'High', 'Low', 'Close']].plot()

In [7]:
hist = hist[['Open', 'High', 'Low', 'Close']]

In [8]:
df = hist.pct_change()

In [9]:
df.plot()

In [10]:
print(len(df) - len(df.dropna()))
df.dropna(inplace=True)

1


In [11]:
df = df[['High']]

In [12]:
x, y = wavy.create_panels(df, lookback=1, gap=0, horizon=1)

In [13]:
x.plot()

Predict always a constant value. From a pct_changed series this model assumes that tomorrow's values will be the same as today's.

In [14]:
baseline = wavy.models.BaselineConstant(x, y, model_type="regression", constant=0)

baseline.fit()
baseline.score()

 85/100 [========================>.....] - ETA: 0s - loss: 0.0000e+00 - mae: 0.0096

2022-06-30 20:05:53.308602: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


100/100 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - mae: 0.0095 - val_loss: 0.0000e+00 - val_mae: 0.0100


,train,test,val
mae,0.00952,0.009665,0.009967


BaselineShift model will predict tomorrow's change exactly as today's change.

In [15]:
shift = wavy.models.BaselineShift(x, y, model_type="regression")

shift.fit()
shift.score()

100/100 [==============================] - 0s 788us/step - loss: 4.0537e-04 - mae: 0.0134 - val_loss: 4.2969e-04 - val_mae: 0.0144


,train,test,val
mae,0.01339,0.012921,0.014412


In [16]:
lr = wavy.models.LinearRegression(x, y)
lr.fit(epochs=30, verbose=False)
lr.score()

,train,test,val
mae,0.009552,0.009565,0.009928


In [53]:
dense = wavy.models.DenseModel(x, y, 'regression', dense_layers=3)
dense.fit(epochs=30, verbose=False)
dense.score()

,train,test,val
mae,0.009469,0.009579,0.009971


In [22]:
for lookback in [3, 12]:
    x, y = wavy.create_panels(df, lookback=lookback, horizon=1)
    lr = wavy.models.ConvModel(x, y, 'regression')
    for epochs in [10, 30, 120, 240]:
        lr.fit(epochs=epochs, verbose=False)
        print(lookback, epochs, lr.score().loc['mae', 'test'])

3 10 0.009644786827266216
3 30 0.009615475311875343
3 120 0.009481372311711311
3 240 0.009594910778105259
12 10 0.009621121920645237
12 30 0.009749040007591248
12 120 0.01058100163936615
12 240 0.011568170040845871


In [55]:
# Classification

In [56]:

log = wavy.models.LogisticRegression(x, yabv)
log.fit(epochs=30, verbose=True)
log.score()

In [61]:
for lookback in [5, 10, 50, 120]:
    x, y = wavy.create_panels(df, lookback=lookback, horizon=1)
    yabv = y > 0
    yabv = yabv.astype(int)
    lr = wavy.models.ConvModel(x, yabv, 'classification')
    lr.fit(epochs=100, verbose=False)
    print(lookback, lr.score().loc['AUC', 'test'])

/Users/rodrigonader/Repositories/logspace/wavy/tutorials/../src/wavy/models.py:534: UserWarning:

Kernel size is not a divisor of lookback.



5 0.5582083463668823


/Users/rodrigonader/Repositories/logspace/wavy/tutorials/../src/wavy/models.py:534: UserWarning:

Kernel size is not a divisor of lookback.



10 0.5274479389190674


/Users/rodrigonader/Repositories/logspace/wavy/tutorials/../src/wavy/models.py:534: UserWarning:

Kernel size is not a divisor of lookback.



50 0.490079402923584
120 0.5015997886657715


In [ ]:
x, y = wavy.create_panels(df, lookback=120, gap=0, horizon=30)

In [ ]:
x[0]

In [ ]:
ymax = y.max() > 0.01

In [ ]:
ymax[0]

In [ ]:
ymax.as_dataframe()

In [ ]:
lr = wavy.models.LinearRegression(x, y)
lr.fit(epochs=30, verbose=False)
lr.score()

In [ ]:
dense = wavy.models.DenseModel(x, y, 'regression')
dense.fit(epochs=3, verbose=True)
dense.score()

In [ ]:
conv = wavy.models.ConvModel(x, y, 'regression')
conv.fit(epochs=3, verbose=True)
conv.score()

In [ ]:
model.predict(x.values)

In [ ]:
y_pred = baseline.predict()

In [ ]:
y.plot()

In [ ]:
y_pred.plot()

In [ ]:
rev_y = reverse_pct_change(y, hist)

In [ ]:
rev_y.plot()

In [ ]:
rev_y_pred = reverse_pct_change(y_pred, hist)

In [ ]:
rev_y_pred.plot()

In [ ]:
# How to interpret that error? What does it mean comparing to the mean/std of the val set?
np.median(baseline.y_val.flatten())

In [ ]:
residuals = baseline.residuals()

In [ ]:
residuals.plot() # Highest error is in train set

In [ ]:
compute_default_scores(x_high, y, 'regression', ['MAE', 'MSE'])

In [ ]:
conv = wavy.ConvModel(x_high, y, model_type="regression", conv_layers=1, conv_filters=32,kernel_size=3, dense_layers=1, dense_units=32)
conv.fit(epochs=20)

# using a cannon ball to kill an ant!

In [ ]:
params=[{'conv_layers': [1, 5, 10], 'conv_filters': [16, 32], 'kernel_size': [2, 3, 7], 'dense_layers': [1], 'dense_units': [32]}, {'dense_layers': 1, 'dense_units': 32}]

models = [ConvModel, DenseModel]

# wavy.GridSearchModel(x, y, model_type, models=, )

In [ ]:
lr.score()

In [ ]:
grid_search --> sklearn

In [ ]:
conv.plot_residuals()

In [ ]:
# TODO: Compare residuals between models

In [ ]:
default_scores = compute_default_scores(x, y, model_type="regression", metrics=["MAE", "MSE"])

In [ ]:
# rev_y = reverse_pct_change(y, hist).as_dataframe()
# rev_y_conv = reverse_pct_change(conv.predict(), hist).as_dataframe()
# rev_y_base = reverse_pct_change(baseline.predict(), hist).as_dataframe()

# rev_y_conv.columns = ['conv_' + col for col in rev_y_conv.columns]
# rev_y_base.columns = ['base_' + col for col in rev_y_base.columns]

In [ ]:
# plot_dataframes([rev_y, rev_y_conv, rev_y_base])

In [ ]:
y_pred[0]

In [ ]:
hist.iloc[10:20]

In [ ]:
y_orig.as_dataframe()

In [ ]:
y.as_dataframe()

In [ ]:
y_orig = y.update(hist.shift())

In [ ]:
y_orig[0]

In [ ]:
y_pred_orig = y_pred.shift() + y_orig

In [ ]:
y_pred_orig.plot()

In [ ]:
y_true = y.test.as_dataframe()
y_baseline = baseline.predict().as_dataframe()
y_lr = lr.predict().as_dataframe()
y_dense = dense.predict().as_dataframe()
y_conv = conv.predict().as_dataframe()

y_true.columns = ['true_' + col for col in y_true.columns]
y_baseline.columns = ['baseline_' + col for col in y_baseline.columns]
y_lr.columns = ['lr_' + col for col in y_lr.columns]
y_dense.columns = ['dense_' + col for col in y_dense.columns]
y_conv.columns = ['conv_' + col for col in y_conv.columns]

plot_dataframes([y_true, y_baseline, y_lr, y_dense, y_conv])

In [ ]:
baseline

In [ ]:
dummy = wavy.ShallowModel(x_high, y, model=DummyRegressor, metrics=[mean_absolute_error], strategy="mean")
dummy.fit()

In [ ]:
rf = wavy.ShallowModel(x, y, model=RandomForestRegressor, metrics=[mean_absolute_error], n_estimators=50)
rf.fit()

In [ ]:
dense = wavy.DenseModel(x, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
# Try the same models but with all input columns
# TODO: Use function to compute default results per model

# lr = wavy.LinearRegression(x, y)
# lr.fit(epochs=20)

In [ ]:

# data1 = {'train': x.train.as_dataframe()["High"],
#         'val': x.val.as_dataframe()["High"],
#         'test': x.test.as_dataframe()["High"]
#         }

# data2 = {'train': x.train.as_dataframe()["Low"],
#         'val': x.val.as_dataframe()["Low"],
#         'test': x.test.as_dataframe()["Low"]
#         }

In [ ]:
aaa = y[-1000:]

s1 = y.train.as_dataframe()['High']
s2 = y.val.as_dataframe()['High']
s3 = y.test.as_dataframe()['High']

In [ ]:
fig = Figure()

In [ ]:
# fig.linebar(s)
# fig.scatter(s)

fig.dotbar(s1)
fig.dotbar(s2, color="blue")
fig.dotbar(s3)

In [ ]:
fig.fig

In [ ]:
fig = PanelFigure()
fig.split_sets(data1)
fig.add_area(data1)
fig.add_dotbar(data2)

In [ ]:
fig.compile()